In [1]:
from preprocess_dataset import load_dataset
import pymc3 as pm
import scipy
import scipy.stats as stats
import theano.tensor as tt
from theano.tensor import _shared
import numpy as np

In [2]:
data, genes, cells = load_dataset()
print("Data shape:", data.shape)

Data shape: (4645, 22712)


In [3]:
def compute_stats(data):
    tmp = np.count_nonzero(data, axis=1)
    print("Average/Min/Max non-zero count for each cell: ", np.mean(tmp), np.min(tmp), np.max(tmp))
    tmp = np.count_nonzero(data, axis=0)
    print("Average/Min/Max non-zero count for each gene: ", np.mean(tmp), np.min(tmp), np.max(tmp))
    tmp = np.max(data, axis=0)
    print("Average/Min/Max max-expression value for each gene: ", np.mean(tmp), np.min(tmp), np.max(tmp))
    print("Number of genes with max-expression of 1:", np.where(tmp==1)[0].shape[0])
    # mean non-zero expression value of each gene
    tmp = np.true_divide(data.sum(0),(data!=0).sum(0))
    print("Average/Min/Max mean non-zero expression value for each gene: ", np.mean(tmp), np.min(tmp), np.max(tmp))
    
def normalize_along_columns(data):
    return (data - data.mean(axis=0)) / data.std(axis=0)

In [4]:
compute_stats(data)
data = normalize_along_columns(data)
compute_stats(data)

Average/Min/Max non-zero count for each cell:  4349.26437029 1310 13154
Average/Min/Max non-zero count for each gene:  889.500396266 1 4642
Average/Min/Max max-expression value for each gene:  162.237847834 1 62129
Number of genes with max-expression of 1: 35
Average/Min/Max mean non-zero expression value for each gene:  12.8953778105 1.0 2317.61788618
Average/Min/Max non-zero count for each cell:  22711.6086114 22709 22712
Average/Min/Max non-zero count for each gene:  4644.91995421 3030 4645
Average/Min/Max max-expression value for each gene:  30.4603784446 5.36922198749 68.1469001496
Number of genes with max-expression of 1: 0
Average/Min/Max mean non-zero expression value for each gene:  3.7043241561e-18 -1.25732339263e-14 1.37069281891e-14


In [7]:
n_clusters = 10
n_data = len(cells)
n_feats = len(genes)
data_tensor = _shared(data)

In [8]:
with pm.Model() as model:
    cluster_weights = pm.Dirichlet('cluster_weights', a=np.ones(n_clusters), shape=n_clusters)
    # ensure all clusters have some points
    weights_min_potential = pm.Potential('weights_min_potential', 
                                         tt.switch(tt.min(cluster_weights) < .1, -np.inf, 0))

    
    # choose a cluster (categorical)
    # each cluster has a list of normals from which the data is chosen from
    
    # little tricky to tune these parameters without normalizing
    cluster_means = pm.Normal('cluster_mean', mu=np.zeros((n_clusters, n_feats)), sd=1, shape=(n_clusters, n_feats))
    

    # for n in range(n_clusters): # break symmetry
    #     order_means_potential = pm.Potential('order_means_potential', tt.switch(sum(mean[n]-mean[n-1]) < 0, -np.inf, 0))

    # latent cluster of each observation
    category = pm.Categorical('category',
                              p=cluster_weights,
                              shape=n_data)

    # likelihood for each observed value
    points = pm.MvNormal('obs',
                       mu=cluster_means[category],
                       tau=np.eye(n_feats),
                       observed=data_tensor)

MemoryError: None